In [1]:
import pandas as pd
pd.set_option("display.max_columns",50)
from time import sleep

from nba_api.stats.static import players
from nba_api.stats.endpoints import playerawards, playercareerstats

In [2]:
#get list of all inactive players; to be used for training/testing
inactives=pd.DataFrame(players.get_inactive_players())
inactives


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4568,1627790,Ante Zizic,Ante,Zizic,False
4569,78647,Jim Zoet,Jim,Zoet,False
4570,78648,Bill Zopf,Bill,Zopf,False
4571,78650,Matt Zunic,Matt,Zunic,False


In [13]:
test_df=inactives.iloc[:5].copy()
test_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [14]:
# This function creates the label variable - whether or not a player is in the HOF
def get_hof(row):
    # sleeping to respect NBA's rate limiting
    sleep(0.5)
    awards = playerawards.PlayerAwards(row["id"]).get_data_frames()[0]
    return "Hall of Fame Inductee" in awards["DESCRIPTION"].values
test_df["HOF"] = test_df.apply(get_hof, axis=1)
test_df

,id,full_name,first_name,last_name,is_active,HOF
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False


In [ ]:
def get_career_stats(row):
    # sleep to respect rate limiting
    sleep(0.5)
    #holds the various dfs-season, career, reg season, playoffs
    totals=playercareerstats.PlayerCareerStats(row["id"]).get_data_frames()
    sleep(0.5)
    avgs=playercareerstats.PlayerCareerStats(row["id"],per_mode36="PerGame").get_data_frames()
    #only return career regular season for now
    return pd.concat([totals[1].iloc[0,3:],avgs[1].iloc[0,5:].add_suffix("PG")])
#playercareerstats.PlayerCareerStats(76003).get_data_frames()[0].columns

In [9]:
ai=inactives[inactives["id"]==947]
ai

,id,full_name,first_name,last_name,is_active
1928,947,Allen Iverson,Allen,Iverson,False


In [30]:
get_career_stats(ai)

GP              914
GS              901
MIN           37577
FGM            8467
FGA           19906
FG_PCT     0.425349
FG3M           1059
FG3A           3383
FG3_PCT    0.313036
FTM            6375
FTA            8168
FT_PCT     0.780485
OREB            745
DREB           2649
REB            3394
AST            5624
STL            1983
BLK             164
TOV            3262
PF             1777
PTS           24368
MIN            41.1
FGM             9.3
FGA            21.8
FG_PCT        0.425
FG3M            1.2
FG3A            3.7
FG3_PCT       0.313
FTM             7.0
FTA             8.9
FT_PCT         0.78
OREB            0.8
DREB            2.9
REB             3.7
AST             6.2
STL             2.2
BLK             0.2
TOV             3.6
PF              1.9
PTS            26.7
Name: 0, dtype: object

In [23]:
pd.concat([test_df,test_df.apply(get_career_stats,axis=1)],axis=1)

,id,full_name,first_name,last_name,is_active,HOF,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False,256,53.0,3200,620,1236,0.501618,0.0,3.0,0.000000,225,321,0.700935,283,563,846,85,71,69,247,484,1465
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False,505,NaN,11023,1769,4138,0.427501,NaN,NaN,NaN,1019,1400,0.727857,503,1124,4065,601,131,205,11,1120,4557
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True,1560,1476.0,57446,15837,28307,0.559473,1.0,18.0,NaN,6712,9304,0.721410,2975,9394,17440,5660,1160,3189,2527,4657,38387
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False,586,336.0,15627,3514,7943,0.442402,474.0,1339.0,0.353996,1051,1161,0.905254,219,868,1087,2079,487,46,963,1106,8553
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False,236,145.0,4808,720,1726,0.417149,18.0,76.0,0.236842,372,529,0.703214,286,490,776,266,184,83,309,485,1830
